In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras import backend as K


In [5]:
class AlexNet:
    @staticmethod
    def build(numChannels, imgRows, imgCols, numClasses, activation="relu", weightsPath=None):
        # sequential initialize the mode
        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        
        #if we are using "last channels", update the input shape
        inputShape = (imgRows, imgCols, numChannels)
        # define the first set of CONV => ACTIVATION => POOL layers
        model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",input_shape=inputShape)) #96=convolution Filter,11=each filter size 11*11
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
        # define the second set of CONV => ACTIVATION => POOL layers
        model.add(Conv2D(filters=256,kernel_size=(11,11),padding="valid"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        #3rd convolution layer
        model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid"))
        model.add(Activation("relu"))
        # 4th Convolutional Layer
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
        model.add(Activation(activation))
        # 5th Convolutional Layer
        model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
        model.add(Activation(activation))
        # Max Pooling
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))
        
        # define the first FC => ACTIVATION layers
        model.add(Flatten())
        model.add(Dense(4096))
        model.add(Activation(activation))
        model.add(Dropout(0.4))
        # define the second FC layer
        # 2nd Fully Connected Layer
        model.add(Dense(4096))
        model.add(Activation("relu"))
        # Add Dropout
        model.add(Dropout(0.4))
        
        # lastly, define the soft-max classifier
        model.add(Dense(numClasses))
        model.add(Activation("softmax"))
        # if a weights path is supplied (inicating that the model was pre-trained), then load the weights
        # return the construction architecture
        return model

In [6]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model = AlexNet.build(numChannels=3, imgRows=227, imgCols=227, numClasses=2)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 384)        

In [ ]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator



train_datagen = ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (227, 227),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')


test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (227, 227),
                                            batch_size = 32,
                                            class_mode = 'categorical')

model.fit_generator(training_set,samples_per_epoch = 8000,nb_epoch = 1,validation_data = test_set,nb_val_samples = 2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/home/marzan/work_env/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/marzan/work_env/local/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=1, steps_per_epoch=250, validation_steps=2000)`


Epoch 1/1
249/250 [============================>.] - ETA: 3s - loss: 0.6949 - accuracy: 0.4910

In [ ]:
# serialize model to JSON
from keras.models import model_from_json
model_json = model.to_json()
with open("model_alexnet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_alexnet_weights.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json
# Model reconstruction from JSON file
with open('model_alexnet.json', 'r') as f:
    model_after = model_from_json(f.read())

# Load weights into the new model
model_after.load_weights('model_alexnet_weights.h5')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
img = cv2.imread("/media/marzan/mini storage/CNN/cat.jpg")

In [ ]:
reshape = cv2.resize(img,(64,64))
dimention_currection = np.reshape(reshape,[1,64,64,3])

In [ ]:
predict = model_after.predict(dimention_currection)
predict